# Case Studies in High-Performance Computing

## Assignment 1 - Communication-Avoiding Methods

### Python Implementation

#### Preliminaries

We first import any necessary packages:

In [1]:
import numpy as np
from scipy.linalg import qr

Moreover, we set a global printing setting below where we only print up to four decimal places in order to make the results easier to visualise:

In [2]:
np.set_printoptions(precision=4, suppress=True)

#### How the Implementation Follows the Mathematical Background

The function below implements the TSQR factorisation as described in `README.md`. Below is a step-by-step breakdown of how the code follows the theory:

1. **Matrix Partitioning**: The function extracts the matrix dimensions `m` and `n`, then determines the `blockSize = m // 4` to divide the matrix into four row blocks. It then iterates over four blocks using a `for` loop and slices the corresponding rows from `W`.
2. **Local QR Factorisation**: Each block `W_i` undergoes a local QR factorisation using `qr(W[rows, :], mode="economic")`. The computed `Q_i` (orthogonal) and `R_i` (upper triangular) are stored in `Q_blocks` and `R_blocks`, respectively.
3. **Reduction Step**: The local `R_i` matrices (each of size $n\times n$) are stacked vertically using `np.vstack(R_blocks)`, forming a $4n\times n$ matrix. Another QR factorisation is performed on this reduced matrix: `Q2, R_final = qr(R_stacked, mode="economic")`. Here, `Q2` is an intermediate orthogonal matrix of size $4n\times n$, and `R_final` is the final upper triangular matrix $n\times n$.
4. **Constructing the Final $Q$**: A zero matrix `Q` of size $m\times n$ is initialised to store the final orthogonal factor. The intermediate `Q2` is partitioned into four sub-matrices corresponding to the four processors (theoretically speaking, since this code merely simulates the communication-avoiding TSQR factorisation by performing computations in sequential steps), and each `Q_i` is updated as `Q_blocks[i] @ Q2_i`. This step combines the local orthogonal matrices `Q_i` with the global reduction matrix `Q2`, resulting in the final `Q`.
5. **Verification**: A random matrix `A` of size $16\times4$ is used for testing. We perform the factorisation, and then the reconstructed matrix `Q @ R` is compared to `A`. The residual norm, given by `A - Q @ R` (i.e., $||A-QR_{\text{final}}||$ is calculated) is printed to check the accuracy of the factorisation.

In [ ]:
def tsqr(W):
    """Compute the tall-skinny QR (TSQR) factorisation of a matrix.

    Args:
        W (numpy.ndarray): A NumPy array (of size m x n), representing the matrix to factorise, where m >> n (i.e., the matrix is tall and skinny).

    Returns:
        tuple:
            - Q (numpy.ndarray): A NumPy array (of size m x n), representing the orthogonal (i.e., Q factor of the decomposition) matrix.
            - R (numpy.ndarray): A NumPy array (of size n x n), representing the upper triangular (i.e., R factor of the decomposition) matrix.
    """

    # Extract matrix dimensions
    m, n = W.shape

    # How many rows each block should have
    blockSize = m // 4

    # Initialise storage for local decompositions
    Q_blocks = []
    R_blocks = []

    # Local QR factorisation loop
    for i in range(4):

        # Extract out the relevant block for the iteration
        rows = slice(i * blockSize, (i + 1) * blockSize)

        # Carry out local QR factorisation
        Q_i, R_i = qr(W[rows, :], mode="economic")

        # Store results
        Q_blocks.append(Q_i)
        R_blocks.append(R_i)

    # Stack the upper triangular matrices vertically
    R_stacked = np.vstack(R_blocks)

    # QR factorisation on the reduced matrix above
    Q2, R_final = qr(R_stacked, mode="economic")

    # Initialise a matrix to store final orthogonal matrix
    Q = np.zeros((m, n))

    # Loop to carry out the multiplication to obtain the final orthogonal matrix
    for i in range(4):

        # Extracting out the relevant blocks
        Q2_i = Q2[i * n : (i + 1) * n, :]

        # Multiplication of old blocks with these new ones
        Q[i * blockSize : (i + 1) * blockSize, :] = Q_blocks[i] @ Q2_i

    return Q, R_final


# Test
m, n = 16, 4
A = np.random.randn(m, n)
Q, R = tsqr(A)
A_reconstructed = Q @ R
residual = np.linalg.norm(A - A_reconstructed)
print("Original matrix, A:")
print(A)
print("\nFinal orthogonal matrix, Q:")
print(Q)
print("\nFinal upper triangular matrix, R:")
print(R)
print("\nReconstructed matrix, QR:")
print(A_reconstructed)
print(f"\nResidual norm, ||A - QR|| = {residual}")

Original matrix, A:
[[-1.3595  0.9003 -0.3804  0.314 ]
 [-0.3389  0.5992  0.1897  0.6584]
 [ 0.0507  0.9738 -2.0994 -1.5068]
 [-1.0978 -1.6898  0.2815 -2.1273]
 [ 0.1333  1.3869  0.9983  0.9044]
 [ 0.5232  1.3155  0.2982  1.1079]
 [ 0.3749 -0.7843  0.5366 -0.8139]
 [-0.414  -1.8155 -0.5376  0.5128]
 [ 0.762   0.3767 -0.3396 -0.7472]
 [-1.0872 -0.7293 -2.1003 -0.0609]
 [ 1.1346 -0.8348  0.1458  0.8893]
 [ 0.3075 -0.1048 -0.3756  1.5401]
 [ 0.9048 -0.2402 -0.1158 -1.2824]
 [ 0.9794 -0.1732 -0.2978 -0.6528]
 [-1.1376 -0.4185 -1.1667 -0.0106]
 [-0.524  -0.0697  1.2095 -3.4765]]

Final orthogonal matrix, Q:
[[ 0.4243  0.315   0.009   0.0639]
 [ 0.1058  0.1795 -0.071   0.1152]
 [-0.0158  0.2596  0.6025 -0.4197]
 [ 0.3426 -0.3966 -0.1938 -0.251 ]
 [-0.0416  0.3664 -0.245   0.1149]
 [-0.1633  0.3264 -0.0199  0.1193]
 [-0.117  -0.2313 -0.1307 -0.1152]
 [ 0.1292 -0.4669  0.0875  0.2177]
 [-0.2378  0.0608  0.1622 -0.2184]
 [ 0.3393 -0.1384  0.4838  0.0266]
 [-0.3541 -0.2854  0.0389  0.1733]
 [-0.